In [ ]:
%load_ext autoreload
%autoreload 2

import os
import shutil

try:
  import google.colab
  os.system("git clone https://github.com/matt-nann/AuthenticCursor.git")
  try:
    shutil.copytree("AuthenticCursor/src", "src")
  except:
    shutil.rmtree("src")
    shutil.copytree("AuthenticCursor/src", "src")
  try:
    shutil.copy("AuthenticCursor/requirementsGAN.txt", "requirementsGAN.txt")
  except:
    shutil.rmtree("requirementsGAN.txt")
    shutil.copy("AuthenticCursor/requirementsGAN.txt", "requirementsGAN.txt")
  # remove conflicting dependencies with google colab preinstalled libraries
  with open("requirementsGAN.txt", "r") as f:
    lines = f.readlines()
    with open("requirementsGAN.txt", "w") as f:
      for line in lines:
        if "numpy" not in line and 'pillow' not in line:
          f.write(line)
  os.system("pip install -r requirementsGAN.txt")
  shutil.rmtree("AuthenticCursor")
  # installing and logging into weights and biases
  os.system("pip install wandb")
  os.system("wandb login")
except Exception as e:
  print(e)

import torch
import wandb # will be prompted for API key in google colab

In [ ]:
from src.mouseGAN.dataProcessing import MouseGAN_Data
from src.mouseGAN.dataset import getDataloader, visuallyVertifyDataloader

USE_FAKE_DATA = True
SAVE_FAKE_DATA = False
RELOAD_FAKE_DATA = True
TRAIN_TEST_SPLIT = 0.8
dataset = MouseGAN_Data(USE_FAKE_DATA=USE_FAKE_DATA, TRAIN_TEST_SPLIT=TRAIN_TEST_SPLIT, 
                        equal_length=False)

SAMPLES = 1000
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False
if USE_FAKE_DATA:
    if RELOAD_FAKE_DATA:
        dataset.createFakeWindMouseDataset(save=SAVE_FAKE_DATA, samples=SAMPLES,
                                        low_radius = 200, high_radius = 1000,
                                        max_width = 300, min_width = 25,
                                        max_height = 300, min_height = 25,)
    else:
        dataset.loadFakeWindMouseData()
else:
    df_moves, df_trajectory = dataset.collectRawMouseTrajectories()

In [ ]:
import time
s_time = time.time()
train_trajs, train_targets, test_trajs, test_targets = dataset.processMouseData(SHOW_ALL=False)
print(f"Time to process data: {time.time() - s_time} seconds")

In [ ]:
from src.mouseGAN.model_config import Config, LR_SCHEDULERS, LOSS_FUNC, \
    C_MiniBatchDisc, C_Discriminator, C_Generator, C_EMA_Plateua_Sch, \
    C_Step_Sch, C_LossGap_Sch
from src.mouseGAN.models import MouseGAN
from src.mouseGAN.experimentTracker import initialize_wandb

# IN_COLAB = True
LOAD_PRETRAINED = False
BATCH_SIZE = 256
num_epochs = 1000
num_feats = train_trajs[0].shape[1]
latent_dim = 100
num_target_feats = 4 # width, height, start_x, start_y
MAX_SEQ_LEN = max([len(traj) for traj in train_trajs + test_trajs])

D_config = C_Discriminator(lr=0.0001, bidirectional=True, hidden_units=128, 
                            num_lstm_layers=4, useEndDeviationLoss=True,
                            gradient_maxNorm = 1.0,)
G_config = C_Generator(lr=0.0001, hidden_units=128, num_lstm_layers=4, useOutsideTargetLoss=True, drop_prob=0.1,
                layer_normalization = True,
                residual_connections = True,
                gradient_maxNorm = 1.0,)

# D_sch_config = C_Step_Sch(2, 0.5)
D_sch_config = C_LossGap_Sch(cooldown=int(BATCH_SIZE)/8, lr_shrinkMin=0.1, lr_growthMax=2.0, 
                            discLossDecay=0.8, lr_max = 0.0005, lr_min = 1*10**(-9))
# G_sch_config = C_Step_Sch(2, 0.5)
# G_sch_config = C_EMA_Plateua_Sch(patience=BATCH_SIZE, cooldown=int(BATCH_SIZE/8), factor=0.5, ema_alpha=0.4)

config = Config(num_epochs, BATCH_SIZE, num_feats, latent_dim, num_target_feats, MAX_SEQ_LEN,
                discriminator=D_config, generator=G_config, 
                D_lr_scheduler=D_sch_config, #G_lr_scheduler=G_sch_config,
                locationMSELoss = False)

## verifying the mean trajectory is centered around zero (even class distribution)
# dataset.plotMeanPath()
trainDataloader = getDataloader(train_trajs, train_targets, config.BATCH_SIZE)
testDataloader = getDataloader(test_trajs, test_targets, config.BATCH_SIZE)

visuallyVertifyDataloader(trainDataloader, dataset, showNumBatches=1)

if IN_COLAB:
    run = initialize_wandb(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gan = MouseGAN(dataset, trainDataloader, testDataloader, device, config, IN_COLAB=IN_COLAB, verbose=True, printBatch=True)
if LOAD_PRETRAINED:
    
    gan.loadPretrained(startingEpoch='final')

print(gan.discriminator)
print(gan.generator)

gan.train(modelSaveInterval=3, catchErrors=False)
if IN_COLAB:
    wandb.finish()

In [ ]:
gan.visualTrainingVerfication(samples=10)

In [ ]:
gan.train(modelSaveInterval=3, catchErrors=False)

In [ ]:
# gan.save_models('final')
gan.loadPretrained(startingEpoch=99)

In [ ]:
for epoch in ['final']:
    gan.loadPretrained(startingEpoch=epoch)
    gan.visualTrainingVerfication()

In [ ]:
# 4000 training samples
print('E1B0 : start of g_loss -> before g_stop_token_loss', 3245.4140625 - 3244.5859375)
print('E1B1 : start of g_loss -> before g_stop_token_loss', 3254.94921875 - 3249.27734375)
print('E1B2 : start of g_loss -> before g_stop_token_loss', 3261.1328125 - 3255.4609375)
print('E1B3 : start of g_loss -> before g_stop_token_loss', 3270.91796875 - 3261.89453125)
print('E1B4 : start of g_loss -> before g_stop_token_loss', 3277.6171875 - 3271.6875)

In [ ]:

epoch:  761144832
	 batch:  0 761144832
d_loss 4.928228855133057 d_loss_real 0.9162368774414062 d_loss_fake 1.0765330791473389 d_loss_dev 3.9318437576293945
g_loss 0.9645814299583435 g_stop_token_loss_length 66.0078125 g_stop_token_loss * lambda_stopLoss 3.3003906250000004
	Batch 1/32, d_loss = 4.928, g_loss = 30.607
	 batch:  1 788964352
d_loss 4.848016738891602 d_loss_real 0.9385039210319519 d_loss_fake 1.0357764959335327 d_loss_dev 3.8608765602111816
g_loss 1.0467503070831299 g_stop_token_loss_length 68.86328125 g_stop_token_loss * lambda_stopLoss 3.4431640625
	Batch 2/32, d_loss = 4.848, g_loss = 31.141
	 batch:  2 788964864
d_loss 5.018941402435303 d_loss_real 0.9796853065490723 d_loss_fake 0.9515709280967712 d_loss_dev 4.053313255310059
g_loss 1.2331814765930176 g_stop_token_loss_length 67.1484375 g_stop_token_loss * lambda_stopLoss 3.357421875
	Batch 3/32, d_loss = 5.019, g_loss = 29.385
	 batch:  3 788964864
d_loss 4.711278438568115 d_loss_real 1.0567747354507446 d_loss_fake 0.8240370750427246 d_loss_dev 3.7708725929260254
g_loss 1.42510187625885 g_stop_token_loss_length 67.98828125 g_stop_token_loss * lambda_stopLoss 3.3994140625
	Batch 4/32, d_loss = 4.711, g_loss = 25.259
	 batch:  4 789058560
d_loss 4.616765022277832 d_loss_real 1.1126352548599243 d_loss_fake 0.6521314382553101 d_loss_dev 3.734381914138794
g_loss 1.7698607444763184 g_stop_token_loss_length 65.2578125 g_stop_token_loss * lambda_stopLoss 3.2628906250000003
	Batch 5/32, d_loss = 4.617, g_loss = 24.820
	 batch:  5 788964864
d_loss 4.494575023651123 d_loss_real 1.1078312397003174 d_loss_fake 0.4396367073059082 d_loss_dev 3.7208409309387207
g_loss 2.6413750648498535 g_stop_token_loss_length 74.1796875 g_stop_token_loss * lambda_stopLoss 3.708984375
	Batch 6/32, d_loss = 4.495, g_loss = 28.195
	 batch:  6 788964864
d_loss 4.501669883728027 d_loss_real 1.3164254426956177 d_loss_fake 0.24257078766822815 d_loss_dev 3.7221715450286865
g_loss 1.6450245380401611 g_stop_token_loss_length 66.55078125 g_stop_token_loss * lambda_stopLoss 3.3275390625
	Batch 7/32, d_loss = 4.502, g_loss = 23.690
	 batch:  7 788964864
d_loss 4.322404861450195 d_loss_real 0.8055024743080139 d_loss_fake 0.5061573386192322 d_loss_dev 3.6665749549865723
g_loss 2.041877269744873 g_stop_token_loss_length 62.91015625 g_stop_token_loss * lambda_stopLoss 3.1455078125
	Batch 8/32, d_loss = 4.322, g_loss = 23.193
	 batch:  8 788964864
d_loss 4.225168228149414 d_loss_real 0.7835075259208679 d_loss_fake 0.37089186906814575 d_loss_dev 3.6479687690734863

In [ ]:
[788964352, 788964864, 788964864, 789058560, 788964864, 788964864, 788964864, 788964864]

In [ ]:
import torch

import os
import psutil

def print_memory_usage(tag):
    process = psutil.Process(os.getpid())
    print(f'{tag} : Memory usage: {process.memory_info().rss / 1024 ** 2} MB')


# Create some tensors
x = torch.randn((10, 10), requires_grad=True)
y = torch.randn((10, 10), requires_grad=True)
graphList = []
with torch.profiler.profile(profile_memory=True, record_shapes=True) as prof:
    for i in range(20):
        print_memory_usage('gi')
        # Run your training step
        z = x + y
        graphList.append(z.sum().backward())

# Print profiler output
print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))
